In [17]:
import pandas as pd
import twint
import nest_asyncio
from datetime import date

def tweet_scrape(search_term):
    nest_asyncio.apply()
    c = twint.Config()
    c.Lang = "en"
    c.Since = str(date.today())
    c.Search = [search_term]
    c.Pandas = True
    twint.run.Search(c)
    Tweets_df = twint.storage.panda.Tweets_df
    return Tweets_df


Tweets_df = tweet_scrape('hydroflask')


1550515844718833664 2022-07-22 09:19:19 -0700 <toddeherman> "Where did Mr. Biden fall down on his bike?"  "I don't think that matters. What does matter is, all dementia-ridden, 79 year-olds who regularly fall down trying to climb stairs should go ride bikes as a photo opportunity."
1550515393881485312 2022-07-22 09:17:31 -0700 <toddeherman> The fact that anyone who works in "news" still takes the words of "public" "health" officials seriously is all the proof anyone needs that we live in a controlled media environment.  https://t.co/HPCQzGGBGK
1550514252221681664 2022-07-22 09:12:59 -0700 <5y42902c54N> 2) Hydroflask (pls) I WANNA PUT CUTE STICKERS ON IT :33333  https://t.co/2adspdyzqx
1550511589941383169 2022-07-22 09:02:24 -0700 <PilipinoTotoo> @YuujiS3nPai Kahit ano pang brand. Better to bring water than purchase single use plastic bottle. Coleman noon Pero ang laki... Mas madaling dalhin yung aqua/hydroflask
1550511391202689024 2022-07-22 09:01:37 -0700 <kwonyuwreee> bumibili siya h

In [18]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def vader_scores(tweet):
    """
    Run VADER sentiment analysis and return  values
    """
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(tweet)
    return list(vs.values())


def score_columns():
    """
    Apply those values to the tweets_df and then seperate them into distinct columns
    """
    Tweets_df['Sentiment'] = Tweets_df['tweet'].map(vader_scores)
    Tweets_df[['negative','neutral','positive', 'compound']] = Tweets_df['Sentiment'].tolist()
    Tweets_df.drop(columns='Sentiment', inplace=True)

score_columns()


In [20]:
from google.cloud import bigquery


def load_bq(dataframe, table):
    client = bigquery.Client()
    df = dataframe
    job = client.load_table_from_dataframe(df, table)
    job.result()


load_bq(Tweets_df,'deb-01-346001.vibe_shift.hydroflask')